<a href="https://colab.research.google.com/github/yassenayman/Question_Answering/blob/main/Haystack_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

## Importing all Packages

In [ ]:
import logging
from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http
from haystack.nodes import TfidfRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pprint import pprint
from haystack.utils import print_answers
import pandas as pd

In [ ]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# In-Memory Document Store
document_store = InMemoryDocumentStore()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


## Read Data

In [ ]:
# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>", "content": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.import_utils:Found data stored in 'data'. Delete this first if you really want to fetch new data.
INFO:haystack.utils.preprocessing:Converting data/summerization.txt


[<Document: {'content': ' Dark Psychological: The Practical Uses and Best Defenses of Psychological Psychological Warfare in Everyday Life. How to Detect and Defend Against Manipulation, Deception, Dark PsychologicalPersuasion, and Covert NLP. How to detect and decipher genuine emotions in others so that you can protect yourself from being emotionally abused and manipulated.\n Growing up, a sweetheart of the neighborhood was a victim of an online scam that left her homeless and heartbroken. She was committed to a psychiatric ward for a few months before she had to be moved to a distant relative. The perpetrators lay the same levels of deceit on the web, but fundamentally, the patterns are the same.\n Dark Psychology is a study of the human condition in relation to the psychological nature of humans to prey on others. The book, Dark Psychology: The Practical Uses and Best Defenses of Psychological Warfare in Everyday Life was born. It is a generally held belief that the human\'s mind in

In [ ]:
retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.nodes.retriever.sparse:Found 1 candidate paragraphs from 1 docs in DB


In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
#pprint(prediction)

In [ ]:
# ...or use a util to simplify the output
#from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to control the level of detail
#print_answers(prediction, details="minimum")

In [ ]:
#print(prediction['answers'][1].answer)

In [ ]:
df = pd.read_csv('Qustions with Summarization.csv')
questions = df['Questions'].tolist()
answers = []
context= []

In [ ]:
for question in questions:
  prediction = pipe.run(
      query=question, params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}}
  )
  temp = []
  temp1= []
  for i in range(len(prediction['answers'])):
    temp.append(prediction['answers'][i].answer)
    temp1.append(prediction['answers'][i].context)
    answers.append(temp)
    context.append(temp1)

In [ ]:
res = [j for i in answers for j in i]
res1 = [j for i in context for j in i]
print(res)
print(res1)

['detect and decipher genuine emotions in others', 'Dark personality traits', 'crime perpetration rates', 'Dark personality traits', 'Dark personality traits', 'the person who is being manipulated is given an illusion of choice', 'Emotions', 'it is not your fault', 'it is not your fault', 'Stephen Richards', 'online scam', 'Beryl Markham', 'dark', 'Machiavellianism', 'strong sense of loss', 'attempted cover-up', 'Dark Psychology', 'psychological', 'emotional scars', 'Beryl Markham', 'psychological', 'emotional scars', 'Beryl Markham', 'lies', 'lies', 'blind optimism', 'you get yourself rooted in a situation that slows you down or stop you from moving forward', 'moving forward', 'it is not your fault', 'money and material wealth', 'depressed', 'Social conditioning', 'emotionally scarred', 'mute', 'the perpetrator', 'commence the healing process', 'you get yourself rooted in a situation that slows you down or stop you from moving forward', 'moving forward', 'comfort that those illusions 

In [ ]:
dataframe = {"Answer":res,
             "Context":res1}
df = pd.DataFrame(dataframe)
df.head()
df.to_csv('ans_cont.csv')


In [ ]:
df1=pd.read_csv('ans_cont.csv')
#df1=df1.drop("Unnamed: 0", axis=1)
df1.head()

,Unnamed: 0,Answer,Context
0,0,detect and decipher genuine emotions in others,"ark PsychologicalPersuasion, and Covert NLP. How to detect and decipher genu..."
1,1,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
2,2,crime perpetration rates,dency that the person would exhibit the traits associated with crime perpetr...
3,3,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
4,4,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...


In [ ]:
df2=pd.read_csv('Qustions with Summarization.csv')
#df1=pd.read_csv('ans_cont.csv')
#frames = [df2, df1]

#result = pd.concat(frames)

In [ ]:
df3 = pd.merge(df2, df1)

In [ ]:
df3.head()

,Unnamed: 0,Questions,Answers T5,Answer,Context
0,0,How can you protect yourself from being emotionally abused and manipulated?,genuine emotions,detect and decipher genuine emotions in others,"ark PsychologicalPersuasion, and Covert NLP. How to detect and decipher genu..."
1,1,What has a high tendency that the person would exhibit traits associated wit...,Dark personality traits,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
2,2,Dark personality traits have a high tendency that the person would exhibit t...,crime perpetration rates higher,crime perpetration rates,dency that the person would exhibit the traits associated with crime perpetr...
3,3,What indicates a person is capable of committing a certain type of offence?,characteristics they display,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
4,4,What indicates a person is capable of committing a certain type of offence?,characteristics they display,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...


In [ ]:
df3.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df3.head(30)

,Questions,Answers T5,Answer,Context
0,How can you protect yourself from being emotionally abused and manipulated?,genuine emotions,detect and decipher genuine emotions in others,"ark PsychologicalPersuasion, and Covert NLP. How to detect and decipher genu..."
1,What has a high tendency that the person would exhibit traits associated wit...,Dark personality traits,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
2,Dark personality traits have a high tendency that the person would exhibit t...,crime perpetration rates higher,crime perpetration rates,dency that the person would exhibit the traits associated with crime perpetr...
3,What indicates a person is capable of committing a certain type of offence?,characteristics they display,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
4,What indicates a person is capable of committing a certain type of offence?,characteristics they display,Dark personality traits,rkophobicpsychology illuminates the dark side of human nature.\n Dark person...
5,What is the main difference between reverse psychology and manipulation?,illusion of choice,the person who is being manipulated is given an illusion of choice,verse psychology and manipulation is that the person who is being manipulate...
6,What type of psychology is manipulation?,reverse psychology,Emotions,"realities that entraps us and on their own, they can be devastating.\n Emot..."
7,What does he say?,"He says it's not about becoming a new person, but becoming the person you al...",it is not your fault,omeone you have been in a relationship with that person. He says it is not y...
8,What does he say?,"He says it's not about becoming a new person, but becoming the person you al...",it is not your fault,omeone you have been in a relationship with that person. He says it is not y...
9,Who wrote a book about mending a relationship?,Stephen Richards,Stephen Richards,n do is look out for the signs I mentioned and keep an open mind.\n Stephen ...


In [ ]:
df3.to_csv('Final_File.csv')